<a href="https://colab.research.google.com/github/featHistamine/RainforcementLearing/blob/main/%20Genome%20Information%20Breed%20Classification%20AI%20Contest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!unzip '/content/drive/MyDrive/open.zip'

Archive:  /content/drive/MyDrive/open.zip
  inflating: sample_submission.csv   
  inflating: snp_info.csv            
  inflating: test.csv                
  inflating: train.csv               


In [3]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
snp_info = pd.read_csv("snp_info.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
train.tail()

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
257,TRAIN_257,0,0,0,2,A G,A G,A A,G A,C C,...,A A,G A,A A,G G,A G,G A,A A,A A,A A,B
258,TRAIN_258,0,0,0,2,G G,A A,C A,A A,A A,...,G A,G A,A A,A G,A G,A A,A G,A A,G A,C
259,TRAIN_259,0,0,0,1,A G,G G,A A,G A,A A,...,G G,G A,G A,A A,G G,G G,G G,C A,G G,A
260,TRAIN_260,0,0,0,1,A A,G G,A A,G A,A A,...,G G,A A,G A,A G,A G,G A,G G,C A,G G,A
261,TRAIN_261,0,0,0,2,G G,A G,C A,G G,C C,...,A A,A A,A A,G G,A A,A A,G G,A A,G A,B


In [103]:
from os import pread
import torch
import torch.nn as nn
import random
from torch import optim
from IPython import display

random.seed(422)
torch.manual_seed(422)

class MLP(torch.nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.flatten = nn.Flatten()
    self.layer = nn.Sequential(
      nn.Linear(160, 20),
      nn.LeakyReLU(),
      nn.Linear(20, 20),
      nn.LeakyReLU(),
      nn.Linear(20, 3)
  )
  def forward(self, x):
    out = self.flatten(x)
    return self.layer(out)

def pretreatment(data):
  import torch
  C = 0
  out = []
  for C in range(data.shape[0]):
    result = list(data.iloc[C, 4 :-1])
    element = list()
    if result[0] == 1: element.append([1,0,0,0,0,0,0,0,0,0])
    else : element.append([0,0,0,0,0,0,0,0,0,0])
    for R in range(len(result)):
      result_R = result[R] 
      if result_R == "A A": element.append([0,1,0,0,0,0,0,0,0,0])
      elif result_R == "G G": element.append([0,0,1,0,0,0,0,0,0,0])
      elif result_R == "C C": element.append([0,0,0,1,0,0,0,0,0,0])
      elif result_R == "A G": element.append([0,0,0,0,1,0,0,0,0,0])
      elif result_R == "G A": element.append([0,0,0,0,0,1,0,0,0,0])
      elif result_R == "A C": element.append([0,0,0,0,0,0,1,0,0,0])
      elif result_R == "C A": element.append([0,0,0,0,0,0,0,1,0,0])
      elif result_R == "G C": element.append([0,0,0,0,0,0,0,0,1,0])
      else:  result.append([0,0,0,0,0,0,0,0,0,1])
      #element = np.array(element)
    out.append(element)
  out = torch.Tensor(out)
  return out

def pretreatment_label(data):
  out = []
  for i in range(data.shape[0]):
    result = []
    if data.iloc[i, -1] == 'A':   result.append([1,0,0])
    elif data.iloc[i, -1] == 'B': result.append([0,1,0])
    else: result.append([0,0,1])
    out += result
  import torch
  out = torch.Tensor(out)
  return out


def train_and_eval(model,train_data, test_data, criterion, optimizer, epoch):
  import matplotlib.pyplot as plt
  X_train = pretreatment(train_data)
  y_train = pretreatment_label(train_data)
  X_test = pretreatment(test_data)
  
  loss_list = []
  epoch_list = []
  for epochs in range(epoch):
    loss = 0.0
    for i in range(X_train.shape[0]):
      input = X_train[i][:][:].view(1, -1)
      label = y_train[i][:][:].view(1, 3)
      output = model(input)
      loss = criterion(output, label)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss += loss.item()
    loss_list += [loss.item()]
    epoch_list += [epochs]
    if (epochs+1) % 10 == 0:
      print(f'Epoch: {epochs+1}, cost: {loss}')
      plt.plot(epoch_list, loss_list, c = "black")
      plt.title("Loss value;According to the number of times of model training")
      plt.xlabel("epoch")
      plt.ylabel("loss")
      plt.show()
      display.clear_output(wait = True)
  #Test
  prediction = []
  for j in range(X_test.shape[0]):
    x = X_test[j][:][:].view(1,-1)
    label_pre = model(x).view(-1)
    label_pre = [label_pre[0], label_pre[1], label_pre[2]]
    if label_pre.index(max(label_pre)) == 0 : prediction += ["A"]
    elif label_pre.index(max(label_pre)) == 1 : prediction += ["B"]
    else: prediction += ["C"]
  print(prediction)
  print(loss_list[-1])

  #for i in range(X_test.shape[0]):

test = pd.read_csv("test.csv")
test["0"] = range(0,175)
test_data = pretreatment(test)

model = MLP()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-7)
train_and_eval(
    model,train, test, criterion, optimizer, 190
               )

['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'B', 'A', 'B', 'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B', 'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B', 'A', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'B', 'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 'B', 'C', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 'C', 'C', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C', 'B', 'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 'B', 'B', 'A', 'A', 'C', 'B', 'B', 'A', 'B', 'C', 'C', 'B', 'B']
9.709744335850701e-05


In [106]:
sample_submission = pd.read_csv("sample_submission.csv")
print(sample_submission)
predict = ['A', 'B', 'C', 'C', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'B', 'A', 'B', 
     'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B',
     'C', 'B', 'B', 'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B', 'A', 'B',
     'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C', 'A', 'B', 'A', 'A', 'B', 'B', 'B',
     'A', 'A', 'C', 'B', 'B', 'B', 'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A',
     'B', 'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'A', 'C', 'B',
     'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B',
     'B', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'A', 'A', 'C', 'B',
     'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C',
     'B', 'B', 'A', 'A', 'C', 'B', 'B', 'A', 'B', 'C', 'C', 'B', 'B'] #0.9719171917
predict2 = ['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'B', 'A', 'B',
      'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B',
      'C', 'B', 'B', 'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B', 'A', 'B',
      'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C', 'A', 'B', 'A', 'A', 'B', 'B', 'B',
      'A', 'A', 'C', 'B', 'C', 'B', 'B', 'B', 'C', 'B', 'A', 'B', 'C', 'C', 'B', 'B', 'C', 'A',
      'B', 'B', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'A', 'C', 'B',
      'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B',
      'B', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C', 'B',
      'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C',
      'B', 'B', 'A', 'A', 'C', 'B', 'B', 'A', 'B', 'C', 'C', 'B', 'B'] #0.9531953195	
predict3 = ['A', 'B', 'C', 'B', 'A', 'B', 'C', 'B', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'B', 'A', 'B',
      'B', 'C', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'A', 'A', 'B', 'C', 'A', 'B',
      'C', 'B', 'B', 'C', 'A', 'B', 'C', 'B', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'B', 'A', 'B', 
      'B', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'C', 'C', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 
      'A', 'A', 'C', 'B', 'C', 'B', 'B', 'B', 'C', 'B', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'A', 
      'B', 'C', 'C', 'B', 'B', 'C', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 
      'A', 'B', 'C', 'B', 'B', 'B', 'B', 'A', 'A', 'C', 'B', 'C', 'A', 'B', 'B', 'C', 'C', 'B', 
      'C', 'C', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C', 'B', 
      'B', 'B', 'A', 'B', 'C', 'A', 'A', 'B', 'A', 'B', 'B', 'A', 'A', 'B', 'C', 'A', 'B', 'C', 
      'B', 'B', 'A', 'A', 'C', 'B', 'B', 'A', 'B', 'C', 'C', 'B', 'B']#0.9622367466

sample_submission["class"] = predict3

sample_submission.to_csv("submission_Histamine03.csv", index = False)

           id class
0    TEST_000     A
1    TEST_001     A
2    TEST_002     A
3    TEST_003     A
4    TEST_004     A
..        ...   ...
170  TEST_170     A
171  TEST_171     A
172  TEST_172     A
173  TEST_173     A
174  TEST_174     A

[175 rows x 2 columns]


In [107]:
from torchsummary import summary
model = MLP()
summary(model, input_size=(160,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 160]               0
            Linear-2                   [-1, 20]           3,220
         LeakyReLU-3                   [-1, 20]               0
            Linear-4                   [-1, 20]             420
         LeakyReLU-5                   [-1, 20]               0
            Linear-6                    [-1, 3]              63
Total params: 3,703
Trainable params: 3,703
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.02
----------------------------------------------------------------


In [ ]:
for param in model.parameters():
  print(param.requires_grad)

True
True
True
True
True
True
True
True
